#Model Inference API
Описание
В этом модуле реализованы функции для инференса (предсказания) ключевых HR-показателей:

Зарплата по вакансии (predict_salary)

Грейд (junior/middle/senior/lead) (predict_grade)

Функции полностью воспроизводят пайплайн, использованный при обучении моделей: учитывают числовые, категориальные и текстовые признаки, а также эмбеддинги описания.

In [ ]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [ ]:
# src/bot/model_inference.py
#%%writefile src/bot/model_inference.py

import os
import joblib
import pickle
import numpy as np
from scipy import sparse
from sentence_transformers import SentenceTransformer

MODEL_DIR = '/content/drive/MyDrive/hh-hr-bot/models'

SALARY_MODEL_PATH = os.path.join(MODEL_DIR, 'salary_lgbm_model.pkl')
GRADE_MODEL_PATH = os.path.join(MODEL_DIR, 'grade_rf_model.joblib')
TFIDF_DESC_PATH = os.path.join(MODEL_DIR, 'tfidf_desc.pkl')
TFIDF_TITLE_PATH = os.path.join(MODEL_DIR, 'tfidf_title.pkl')
OHE_CATS_PATH = os.path.join(MODEL_DIR, 'ohe_cats.pkl')

salary_model = joblib.load(SALARY_MODEL_PATH)
grade_model = joblib.load(GRADE_MODEL_PATH)

with open(TFIDF_DESC_PATH, 'rb') as f:
    tfidf_desc = pickle.load(f)
with open(TFIDF_TITLE_PATH, 'rb') as f:
    tfidf_title = pickle.load(f)
with open(OHE_CATS_PATH, 'rb') as f:
    ohe = pickle.load(f)

minilm_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def prepare_features_full(features: dict):
    """ Полный пайплайн для salary (668 признаков) """
    num_feats = np.array([
        features.get('area_id', 0),
        features.get('desc_len', 0),
        features.get('desc_words', 0),
        features.get('title_len', 0),
        features.get('num_skills', 0),
        features.get('exp_junior', 0),
        features.get('exp_middle', 0),
        features.get('exp_senior', 0),
        features.get('exp_lead', 0)
    ]).reshape(1, -1)

    desc = features.get('description', "")
    tfidf_desc_vec = tfidf_desc.transform([desc])
    title = features.get('title', "")
    tfidf_title_vec = tfidf_title.transform([title])
    area_id = str(features.get('area_id', ""))
    salary_currency = str(features.get('salary_currency', "RUR"))
    ohe_cats_vec = ohe.transform([[area_id, salary_currency]])
    emb_vec = minilm_model.encode([desc])  # (1, 384)
    X = sparse.hstack([
        num_feats, tfidf_desc_vec, tfidf_title_vec, ohe_cats_vec, emb_vec
    ]).tocsr()
    return X

def prepare_features_emb_only(features: dict):
    """ Только эмбеддинг по описанию для grade-классификатора (384 признака) """
    desc = features.get('description', "")
    emb_vec = minilm_model.encode([desc])  # (1, 384)
    return emb_vec

def predict_salary(features: dict) -> float:
    X = prepare_features_full(features)
    salary_pred = salary_model.predict(X)[0]
    return float(salary_pred)

def predict_grade(features: dict) -> int:
    X = prepare_features_emb_only(features)
    grade_pred = grade_model.predict(X)[0]
    return int(grade_pred)

def predict_salary_response(features: dict) -> dict:
    value = predict_salary(features)
    return {
        "salary": int(value),
        "currency": features.get("salary_currency", "RUR")
    }

def predict_grade_response(features: dict) -> dict:
    code = predict_grade(features)
    grade_map = {0: "junior", 1: "middle", 2: "senior", 3: "lead"}
    return {
        "grade_code": int(code),
        "grade_label": grade_map.get(code, "unknown")
    }

if __name__ == "__main__":
    test_features = {
        'area_id': 2,
        'desc_len': 1200,
        'desc_words': 100,
        'title_len': 30,
        'num_skills': 2,
        'exp_junior': 0,
        'exp_middle': 0,
        'exp_senior': 0,
        'exp_lead': 0,
        'description': "Инженер, опыт с Django и PostgreSQL, удалёнка.",
        'title': "Инженер",
        'salary_currency': "RUR"
    }
    print("Зарплата:", predict_salary_response(test_features))
    print("Грейд:", predict_grade(test_features))



Зарплата: {'salary': 158478, 'currency': 'RUR'}
Грейд: 1


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



#market_analytics API

Аналитические функции для HR-бота и автоматизации анализа рынка труда.

Содержит:
- top_5_skills: топ-5 востребованных навыков с фильтрацией по профессии, региону и грейду (опционально), а также средней зарплатой по вакансиям с этим навыком.
- compare_vacancy_to_market: сравнение вакансии с рынком по зарплате и ключевым навыкам (по аналогичным вакансиям из базы).

Все функции работают напрямую с базой hh.duckdb_3000 и могут использоваться в ботах, API, скриптах и презентациях.
"""


In [ ]:
# src/bot/market_analytics.py
%%writefile src/bot/market_analytics.py
import duckdb
import pandas as pd

# Подключение к базе (укажи свой путь, если другой)
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

# Карта грейдов для удобной фильтрации по “человеческим” грейдам
grade_map = {
    'Нет опыта': 0,          # junior
    'От 1 года до 3 лет': 1, # middle
    'От 3 до 6 лет': 2,      # senior
    'Более 6 лет': 3         # lead
}

def top_5_skills(title=None, area_id=None, grade=None):
    """
    Возвращает топ-5 навыков по частоте и средней зарплате.
    Фильтры:
      - title (str): фильтр по профессии (подстрока в названии вакансии)
      - area_id (int): фильтр по региону
      - grade (int/str): фильтр по грейду (0-3 или "Нет опыта", ...)
    """
    # Преобразуем grade к строке на русском, если нужно
    experience_hh = None
    if grade is not None:
        if isinstance(grade, int):
            for k, v in grade_map.items():
                if v == grade:
                    experience_hh = k
                    break
        elif grade in grade_map:
            experience_hh = grade
        else:
            raise ValueError("grade должен быть int (0-3) или одним из: " + ", ".join(grade_map.keys()))

    query = """
    SELECT
        vs.skill_name,
        COUNT(*) AS frequency,
        ROUND(AVG(vp.salary_rub), 0) AS mean_salary
    FROM vacancy_skill vs
    JOIN vacancy v ON vs.vacancy_id = v.id
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE 1=1
    """
    params = []
    if title:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{title.lower()}%")
    if area_id is not None:
        query += " AND v.area_id = ?"
        params.append(area_id)
    if experience_hh is not None:
        query += " AND v.experience_hh = ?"
        params.append(experience_hh)
    query += """
    GROUP BY vs.skill_name
    ORDER BY frequency DESC
    LIMIT 5
    """
    return con.execute(query, params).df()

def compare_vacancy_to_market(vac: dict):
    """
    Сравнивает переданную вакансию (dict) с рынком аналогичных вакансий:
      - title (str)
      - area_id (int)
      - experience_hh (str, например, 'От 1 года до 3 лет')
      - skills (list или строка через ;)
      - salary_rub (int/float, опционально)
    Возвращает текстовый отчёт с анализом зарплаты и навыков.
    """
    title = vac['title']
    area_id = vac['area_id']
    experience_hh = vac['experience_hh']
    skills = vac['skills']
    salary = vac.get('salary_rub', None)

    # === 1. Формируем "аналогичный рынок" вакансий ===
    query = """
    SELECT v.id, v.title, v.area_id, v.experience_hh, vp.salary_rub
    FROM vacancy v
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE v.area_id = ?
      AND v.experience_hh = ?
      AND LOWER(v.title) LIKE ?
      AND vp.salary_rub IS NOT NULL
    """
    params = [area_id, experience_hh, f"%{title.lower()}%"]
    df_market = con.execute(query, params).fetchdf()

    if df_market.empty:
        return "Нет сопоставимых вакансий для сравнения."

    # === 2. Медианная зарплата по рынку ===
    market_salary = df_market['salary_rub'].median()
    result = ""
    if salary:
        diff = salary - market_salary
        perc = round(100 * diff / market_salary, 1)
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Зарплата: {int(salary)} руб.\n"
            f"Медиана рынка: {int(market_salary)} руб.\n"
            f"Отклонение: {'+' if diff > 0 else ''}{int(diff)} руб. "
            f"({'+' if perc > 0 else ''}{perc}%)\n"
        )
    else:
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Медиана зарплаты по рынку: {int(market_salary)} руб.\n"
        )

    # === 3. Анализ навыков ===
    # BinderError fix: явно приводим id к BIGINT
    q_skills = """
    SELECT vs.skill_name
    FROM vacancy_skill vs
    WHERE vs.vacancy_id IN (
        SELECT CAST(v.id AS BIGINT)
        FROM vacancy v
        WHERE v.area_id = ?
          AND v.experience_hh = ?
          AND LOWER(v.title) LIKE ?
    )
    """
    skills_df = con.execute(q_skills, params).fetchdf()

    if not skills_df.empty:
        top_skills = (
            skills_df['skill_name']
            .value_counts()
            .head(10)
            .index
            .tolist()
        )
        if isinstance(skills, str):
            your_skills = [s.strip() for s in skills.split(';') if s.strip()]
        else:
            your_skills = list(skills)
        common = [s for s in your_skills if s in top_skills]
        unique = [s for s in your_skills if s not in top_skills]
        missing = [s for s in top_skills if s not in your_skills]
        result += (
            f"\nВаши навыки: {', '.join(your_skills)}\n"
            f"Топ-10 популярных навыков на рынке: {', '.join(top_skills)}\n"
        )
        if unique:
            result += f"Уникальные для вас навыки (редко встречаются): {', '.join(unique)}\n"
        if missing:
            result += f"Рекомендуется добавить популярные навыки: {', '.join(missing)}\n"
    return result

#получение города
def get_area_id_by_city(city_name):
    """
    Возвращает area_id по названию города (или None, если не найдено).
    Поиск регистронезависимый и по подстроке.
    """
    query = "SELECT area_id FROM cities WHERE LOWER(area_name) LIKE ? LIMIT 1"
    result = con.execute(query, [f"%{city_name.lower()}%"]).fetchdf()
    if not result.empty:
        return int(result.iloc[0]['area_id'])
    else:
        return None

def top_vacancies(area_name=None, keyword=None, grade=None, limit=5):
    """
    Возвращает топ-вакансий по зарплате с учётом фильтров.
    area_name: (str) — название города или региона (например, 'Москва')
    keyword: (str) — ключевое слово в названии вакансии (например, 'python')
    grade: (int/str) — грейд (например, 1 или 'От 1 года до 3 лет')
    limit: (int) — сколько вакансий показать
    """
    # Поиск area_id по названию региона
    #area_id = None
    #if area_name:
    #    area_query = "SELECT area_id FROM cities WHERE LOWER(name) LIKE ? LIMIT 1"
    #    df_area = con.execute(area_query, [f"%{area_name.lower()}%"]).fetchdf()
    #    if not df_area.empty:
    #       area_id = int(df_area.iloc[0]['id'])
    #    else:
    #       area_id = None
    #
    area_id = get_area_id_by_city(area_name) or 0

    # Фильтр по грейду
    experience_hh = None
    if grade is not None:
        if isinstance(grade, int):
            for k, v in grade_map.items():
                if v == grade:
                    experience_hh = k
                    break
        elif grade in grade_map:
            experience_hh = grade
        else:
            experience_hh = grade  # Вдруг ввели текст напрямую

    query = """
    SELECT v.title, v.employer, v.area_id, v.experience_hh, vp.salary_rub
    FROM vacancy v
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE vp.salary_rub IS NOT NULL
    """
    params = []
    if area_id:
        query += " AND v.area_id = ?"
        params.append(area_id)
    if keyword:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{keyword.lower()}%")
    if experience_hh:
        query += " AND v.experience_hh = ?"
        params.append(experience_hh)
    query += """
    ORDER BY vp.salary_rub DESC
    LIMIT ?
    """
    params.append(limit)

    return con.execute(query, params).fetchdf()


def promotion_skills(title=None, area_id=None, grade_from=None, grade_to=None, top_n=7):
    """
    Возвращает топ-навыки, которые чаще встречаются у grade_to, чем у grade_from,
    то есть навыки для карьерного роста.
    """
    if grade_from is None or grade_to is None:
        return []

    grade_map = {'junior': 0, 'middle': 1, 'senior': 2, 'lead': 3}
    grades = {v: k for k, v in grade_map.items()}
    from_grade = grades.get(grade_from, grade_from)
    to_grade = grades.get(grade_to, grade_to)

    query = """
    SELECT vs.skill_name, COUNT(*) as freq
    FROM vacancy_skill vs
    JOIN vacancy v ON vs.vacancy_id = v.id
    WHERE 1=1
    """
    params = []
    if title:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{title.lower()}%")
    if area_id:
        query += " AND v.area_id = ?"
        params.append(area_id)

    query_from = query + " AND v.experience_hh = ? GROUP BY vs.skill_name"
    params_from = params + [from_grade]
    df_from = con.execute(query_from, params_from).fetchdf()

    query_to = query + " AND v.experience_hh = ? GROUP BY vs.skill_name"
    params_to = params + [to_grade]
    df_to = con.execute(query_to, params_to).fetchdf()

    skills_from = df_from.set_index('skill_name')['freq'] if not df_from.empty else pd.Series(dtype=int)
    skills_to = df_to.set_index('skill_name')['freq'] if not df_to.empty else pd.Series(dtype=int)

    skill_delta = []
    for skill in skills_to.index:
        freq_to = skills_to[skill]
        freq_from = skills_from[skill] if skill in skills_from else 0
        diff = freq_to - freq_from
        if diff > 0:
            skill_delta.append((skill, diff, freq_to))
    skill_delta.sort(key=lambda x: (-x[1], -x[2]))
    return skill_delta[:top_n]


Overwriting src/bot/market_analytics.py


In [ ]:
import duckdb

# Подключение к базе
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

# Карта грейдов для удобства фильтрации (если нужно фильтровать по коду)
grade_map = {
    'Нет опыта': 0,          # junior
    'От 1 года до 3 лет': 1, # middle
    'От 3 до 6 лет': 2,      # senior
    'Более 6 лет': 3         # lead
}

def top_5_skills(title=None, area_id=None, grade=None):
    """
    Топ-5 навыков по частоте и средней зарплате. Фильтры:
    - title: подстрока в названии профессии
    - area_id: регион
    - grade: int-код (0,1,2,3) или текст на русском ("Нет опыта", ...)
    """
    # Если grade задан как int — подбираем строку для фильтра
    experience_hh = None
    if grade is not None:
        # Можно передать как int или как строку на русском
        if isinstance(grade, int):
            for k, v in grade_map.items():
                if v == grade:
                    experience_hh = k
                    break
        elif grade in grade_map:
            experience_hh = grade
        else:
            raise ValueError("grade должен быть int (0-3) или одним из: " + ", ".join(grade_map.keys()))

    query = """
    SELECT
        vs.skill_name,
        COUNT(*) AS frequency,
        ROUND(AVG(vp.salary_rub), 0) AS mean_salary
    FROM vacancy_skill vs
    JOIN vacancy v ON vs.vacancy_id = v.id
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE 1=1
    """
    params = []
    if title:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{title.lower()}%")
    if area_id is not None:
        query += " AND v.area_id = ?"
        params.append(area_id)
    if experience_hh is not None:
        query += " AND v.experience_hh = ?"
        params.append(experience_hh)
    query += """
    GROUP BY vs.skill_name
    ORDER BY frequency DESC
    LIMIT 5
    """
    return con.execute(query, params).df()

# Примеры вызова:
# Топ-5 по Python, регион 3, грейд "senior" (можно и по коду, и по строке)
print(top_5_skills(title="Инженер", area_id=2, grade=1))             # senior
print(top_5_skills(title="Python", area_id=3, grade="От 3 до 6 лет"))




               skill_name  frequency  mean_salary
0  Проектная документация          2     317500.0
1          Проектирование          1     135000.0
2          Разработка РЭА          1     135000.0
3                  Python          1          NaN
4                    VoIP          1     120000.0
Empty DataFrame
Columns: [skill_name, frequency, mean_salary]
Index: []


In [ ]:
import duckdb

# Подключение к базе DuckDB (путь подставь свой)
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

def compare_vacancy_to_market(vac: dict):
    """
    Сравнивает переданную вакансию (dict) с рынком аналогичных вакансий:
    - title: профессия (str)
    - area_id: регион (int)
    - experience_hh: грейд (str, например, 'От 1 года до 3 лет')
    - skills: список или строка (навыки через ;)
    - salary_rub: (float/int, желательно для отчёта, но не обязательно)

    Возвращает текстовый отчёт с анализом зарплаты и навыков.
    """

    # Извлекаем данные из входного словаря
    title = vac['title']
    area_id = vac['area_id']
    experience_hh = vac['experience_hh']
    skills = vac['skills']
    salary = vac.get('salary_rub', None)  # Может отсутствовать

    # === 1. Формируем аналогичный "рынок" вакансий ===
    # Выбираем из базы вакансии с тем же регионом, грейдом и похожим названием,
    # которые имеют зарплату (salary_rub)
    query = """
    SELECT v.id, v.title, v.area_id, v.experience_hh, vp.salary_rub
    FROM vacancy v
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE v.area_id = ?
      AND v.experience_hh = ?
      AND LOWER(v.title) LIKE ?
      AND vp.salary_rub IS NOT NULL
    """
    params = [area_id, experience_hh, f"%{title.lower()}%"]
    df_market = con.execute(query, params).fetchdf()

    # Если подходящих вакансий нет — сразу возвращаем сообщение
    if df_market.empty:
        return "Нет сопоставимых вакансий для сравнения."

    # === 2. Считаем медиану зарплаты по рынку ===
    market_salary = df_market['salary_rub'].median()

    # Формируем текст отчёта о зарплате
    result = ""
    if salary:
        # Если у пользователя есть своя зарплата — сравниваем с рынком
        diff = salary - market_salary
        perc = round(100 * diff / market_salary, 1)
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Зарплата: {int(salary)} руб.\n"
            f"Медиана рынка: {int(market_salary)} руб.\n"
            f"Отклонение: {'+' if diff > 0 else ''}{int(diff)} руб. "
            f"({'+' if perc > 0 else ''}{perc}%)\n"
        )
    else:
        # Если зарплата не указана — просто сообщаем медиану рынка
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Медиана зарплаты по рынку: {int(market_salary)} руб.\n"
        )

    # === 3. Анализ навыков ===
    # Находим топ-10 навыков среди аналогичных вакансий (по id)
    q_skills = """
    SELECT vs.skill_name
    FROM vacancy_skill vs
    WHERE vs.vacancy_id IN (
        SELECT CAST(v.id AS BIGINT)
        FROM vacancy v
        WHERE v.area_id = ?
          AND v.experience_hh = ?
          AND LOWER(v.title) LIKE ?
    )
    """
    skills_df = con.execute(q_skills, params).fetchdf()

    if not skills_df.empty:
        # Находим 10 самых частых навыков на рынке (по аналогичным вакансиям)
        top_skills = (
            skills_df['skill_name']
            .value_counts()
            .head(10)
            .index
            .tolist()
        )
        # Приводим твои навыки к списку
        if isinstance(skills, str):
            your_skills = [s.strip() for s in skills.split(';') if s.strip()]
        else:
            your_skills = list(skills)
        # Сравниваем: какие твои навыки есть на рынке (common), какие уникальны (unique)
        common = [s for s in your_skills if s in top_skills]
        unique = [s for s in your_skills if s not in top_skills]
        # Какие популярные навыки рынка отсутствуют у тебя (missing)
        missing = [s for s in top_skills if s not in your_skills]
        # Добавляем информацию о навыках в отчёт
        result += (
            f"\nВаши навыки: {', '.join(your_skills)}\n"
            f"Топ-10 популярных навыков на рынке: {', '.join(top_skills)}\n"
        )
        if unique:
            result += f"Уникальные для вас навыки (редко встречаются): {', '.join(unique)}\n"
        if missing:
            result += f"Рекомендуется добавить популярные навыки: {', '.join(missing)}\n"

    # Возвращаем готовый отчёт
    return result

# Пример использования:
my_vac = {
    'title': 'Инженер',
    'area_id': 2,
    'experience_hh': 'От 1 года до 3 лет',
    'skills': ['Python', 'SQL', 'Django', 'English'],
    'salary_rub': 120000
}
print(compare_vacancy_to_market(my_vac))


Ваша вакансия: Инженер (От 1 года до 3 лет, регион 2)
Зарплата: 120000 руб.
Медиана рынка: 142500 руб.
Отклонение: -22500 руб. (-15.8%)

Ваши навыки: Python, SQL, Django, English
Топ-10 популярных навыков на рынке: Проектная документация, Linux, VoIP, Web тестирование, Python, Разработка технических заданий, Разработка РЭА, DesignerСхемотехника электронного оборудования, Чтение электросхем, Проектирование
Уникальные для вас навыки (редко встречаются): SQL, Django, English
Рекомендуется добавить популярные навыки: Проектная документация, Linux, VoIP, Web тестирование, Разработка технических заданий, Разработка РЭА, DesignerСхемотехника электронного оборудования, Чтение электросхем, Проектирование



In [2]:
pip install python-telegram-bot --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 32.9 MB/s eta 0:00:00


In [ ]:
pip install python-dotenv

In [ ]:
%%writefile src/bot/bot.py
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes,MessageHandler, filters, ConversationHandler
from model_inference import predict_salary, predict_salary_response, predict_grade
from telegram.constants import ParseMode
from market_analytics import top_5_skills, compare_vacancy_to_market, top_vacancies, get_area_id_by_city, promotion_skills
import pandas as pd
from dotenv import load_dotenv
import os


SALARY_INPUT = 1

SKILLS_INPUT = 2

ANALYZE_INPUT = 3

TOPJOBS_INPUT = 4

GRADE_INPUT = 5

MAIN_MENU_TEXT = (
    "Вы можете воспользоваться следующими командами:\n"
    "/salary — прогноз зарплаты по вакансии\n"
    "/skills — топ-5 навыков по вакансии\n"
    "/analyze — сравнить вакансию с рынком\n"
    "/top — топ-вакансий по фильтрам\n"
    "/grade — определить грейд\n"
    "/help — справка"
)

# Загрузить переменные окружения из .env
load_dotenv(dotenv_path="/content/drive/MyDrive/hh-hr-bot/.env")

# Получить токен из переменных окружения
TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    # Отправляем картинку
    await update.message.reply_photo(
        photo=open('src/bot/baner.jpg', 'rb'),  # Укажи имя файла-баннера (jpg/png)
        caption="👋 Добро пожаловать в HR-бот!\n\nМогу предсказать зарплату, грейд, показать топ навыков и сравнить вакансию с рынком.\n\n"
        "📌Команды:\n"
        "/salary — прогноз зарплаты\n"
        "/grade — определение грейда\n"
        "/skills — топ-5 востребованных навыков\n"
        "/analyze — сравнить вакансию с рынком\n"
        "/nextskills — Навыки для карьерного роста\n"
        "/top — ТОП вакансий с рынком\n"
        "\nДля справки введите /help"
    )
    #await update.message.reply_text(
    #    "👋 Привет! Я HR-бот. Могу предсказать зарплату, грейд, показать топ навыков и сравнить вакансию с рынком.\n"
    #    "Команды:\n"
    #    "/salary — прогноз зарплаты\n"
    #    "/grade — определение грейда\n"
    ##    "/skills — топ-5 востребованных навыков\n"
     #   "/analyze — сравнить вакансию с рынком\n"
    #    "/nextskills — Навыки для карьерного роста\n"
    ##    "/top — ТОП вакансий с рынком\n"
     #   "\nДля справки введите /help"
    #)

# Состояния
SALARY_DESC, SALARY_CITY, SALARY_SKILLS, SALARY_GRADE = range(4)

async def salary_start(update, context):
    await update.message.reply_text(
        "Опишите вакансию (например: Python-разработчик, поддержка веб-сервиса, автоматизация бизнес-процессов):"
    )
    return SALARY_DESC

async def salary_get_city(update, context):
    context.user_data['description'] = update.message.text.strip()
    await update.message.reply_text("В каком городе или регионе расположена вакансия?")
    return SALARY_CITY

async def salary_get_skills(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "Перечислите ключевые навыки через запятую (например: Python, Django, SQL, коммуникабельность):"
    )
    return SALARY_SKILLS

async def salary_get_grade(update, context):
    context.user_data['skills'] = [s.strip() for s in update.message.text.split(',')]
    await update.message.reply_text(
        "Укажите уровень вакансии (junior, middle, senior, lead).\n"
        "Если не знаете — напишите ‘-’ или оставьте пустым."
    )
    return SALARY_GRADE

async def salary_finish(update, context):
    grade_input = update.message.text.strip().lower()
    grade = grade_input if grade_input in ['junior', 'middle', 'senior', 'lead'] else None
    context.user_data['grade'] = grade

    # --- Готовим данные для модели ---
    description = context.user_data['description']
    city = context.user_data['city']
    skills = context.user_data['skills']
    grade = context.user_data['grade']

    # Найти area_id по названию города (твоя таблица area)

    area_id = get_area_id_by_city(city) or 0


    # Кодируем грейд
    exp_junior = int(grade == 'junior') if grade else 0
    exp_middle = int(grade == 'middle') if grade else 0
    exp_senior = int(grade == 'senior') if grade else 0
    exp_lead = int(grade == 'lead') if grade else 0

    # Остальные признаки
    desc_len = len(description)
    desc_words = len(description.split())
    title_len = min(desc_len, 40)
    num_skills = len(skills)

    features = {
        'area_id': area_id,
        'desc_len': desc_len,
        'desc_words': desc_words,
        'title_len': title_len,
        'num_skills': num_skills,
        'exp_junior': exp_junior,
        'exp_middle': exp_middle,
        'exp_senior': exp_senior,
        'exp_lead': exp_lead,
        'description': description,
        'title': description[:40],
        'salary_currency': "RUR"
    }
    salary = predict_salary(features)
    msg = (
        f"<b>Прогнозируемая зарплата по вакансии:</b> {int(salary):,}".replace(',', ' ') +f" {features['salary_currency']}\n\n"
        f"Ваши данные:\n"
        f"Город: {city} (area_id: {area_id})\n"
        f"Навыки: {', '.join(skills)}\n"
        f"Уровень: {grade if grade else 'не указан'}"
    )
    await update.message.reply_text(msg, parse_mode="HTML")
    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END

GRADE_DESC, GRADE_CITY, GRADE_SKILLS = range(3)

async def grade_start(update, context):
    await update.message.reply_text(
        "Опишите вакансию или свой опыт работы (например: Python-разработчик, поддержка веб-сервиса, автоматизация бизнес-процессов):"
    )
    return GRADE_DESC

async def grade_get_city(update, context):
    context.user_data['description'] = update.message.text.strip()
    await update.message.reply_text("В каком городе или регионе расположена вакансия (или где вы работаете)?")
    return GRADE_CITY

async def grade_get_skills(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "Перечислите ключевые навыки через запятую (например: Python, Django, SQL, коммуникабельность):"
    )
    return GRADE_SKILLS

async def grade_finish(update, context):
    description = context.user_data['description']
    city = context.user_data['city']
    skills = [s.strip() for s in update.message.text.split(',')]

    area_id = get_area_id_by_city(city) or 0
    desc_len = len(description)
    desc_words = len(description.split())
    title = description[:40]
    title_len = len(title)
    num_skills = len(skills)

    # exp_* признаки все по 0 — модель сама определяет грейд
    features = {
        'area_id': area_id,
        'desc_len': desc_len,
        'desc_words': desc_words,
        'title_len': title_len,
        'num_skills': num_skills,
        'exp_junior': 0,
        'exp_middle': 0,
        'exp_senior': 0,
        'exp_lead': 0,
        'description': description,
        'title': title,
        'salary_currency': "RUR"
    }

    grade_code = predict_grade(features)
    grade_map = {0: "junior", 1: "middle", 2: "senior", 3: "lead"}
    grade_label = grade_map.get(grade_code, "unknown")

    msg = (
        f"<b>Определённый грейд:</b> <b>{grade_label.title()}</b> ({grade_code})\n\n"
        f"Ваши данные:\n"
        f"Город: {city} (area_id: {area_id})\n"
        f"Навыки: {', '.join(skills)}\n"
        f"Описание: {description[:40]}..."
    )
    await update.message.reply_text(msg, parse_mode="HTML")
    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END

SKILLS_TITLE, SKILLS_CITY, SKILLS_GRADE = range(3)

async def skills_start(update, context):
    await update.message.reply_text(
        "Введите профессию или краткое название вакансии (например: Python-разработчик, аналитик данных, менеджер по продажам):"
    )
    return SKILLS_TITLE

async def skills_get_city(update, context):
    context.user_data['title'] = update.message.text.strip()
    await update.message.reply_text("В каком городе или регионе вас интересует рынок вакансий?")
    return SKILLS_CITY

async def skills_get_grade(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "Укажите уровень (junior, middle, senior, lead). Если не важен — напишите ‘-’ или оставьте пустым."
    )
    return SKILLS_GRADE

async def skills_finish(update, context):
    grade_input = update.message.text.strip().lower()
    title = context.user_data['title']
    city = context.user_data['city']
    grade = grade_input if grade_input in ['junior', 'middle', 'senior', 'lead'] else None

    # Получить area_id
    area_id = get_area_id_by_city(city) or 0

    # Преобразовать грейд в нужный формат для top_5_skills (int или str)
    grade_map = {'junior': 0, 'middle': 1, 'senior': 2, 'lead': 3}
    grade_code = grade_map.get(grade, None) if grade else None

    # Получить топ-5 навыков
    df = top_5_skills(title=title, area_id=area_id, grade=grade_code)

    if df.empty:
        await update.message.reply_text("Нет данных для выбранных параметров.")
        return ConversationHandler.END

    # Формируем красивый ответ
    msg = f"<b>Топ-5 навыков по запросу:</b>\nПрофессия: {title}\nГород: {city}\n"
    if grade:
        msg += f"Уровень: {grade}\n"
    msg += "\n"
    for i, row in df.iterrows():
        salary = f"{int(row['mean_salary']):,}".replace(',', ' ') if row['mean_salary'] else "-"
        msg += (
            f"{i+1}. <b>{row['skill_name']}</b> — {row['frequency']} вакансий, "
            f"ср. зарплата: {salary} руб.\n"
        )
    await update.message.reply_text(msg, parse_mode="HTML")
    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END

NEXTSKILLS_TITLE, NEXTSKILLS_CITY, NEXTSKILLS_FROM, NEXTSKILLS_TO = range(4)

async def nextskills_start(update, context):
    await update.message.reply_text(
        "Для какой профессии вы хотите узнать навыки для карьерного роста? (например: Python-разработчик)"
    )
    return NEXTSKILLS_TITLE

async def nextskills_get_city(update, context):
    context.user_data['title'] = update.message.text.strip()
    await update.message.reply_text("Укажите город или регион (или ‘-’ если не важно):")
    return NEXTSKILLS_CITY

async def nextskills_get_from(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "С какого грейда хотите начать (junior, middle, senior)?"
    )
    return NEXTSKILLS_FROM

async def nextskills_get_to(update, context):
    context.user_data['grade_from'] = update.message.text.strip().lower()
    await update.message.reply_text(
        "На какой грейд хотите перейти (middle, senior, lead)?"
    )
    return NEXTSKILLS_TO

async def nextskills_finish(update, context):
    grade_to = update.message.text.strip().lower()
    title = context.user_data['title']
    city = context.user_data['city']
    grade_from = context.user_data['grade_from']

    # Получить area_id
    area_id = get_area_id_by_city(city) if city and city != '-' else None

    # Аналитика
    result = promotion_skills(title=title, area_id=area_id, grade_from=grade_from, grade_to=grade_to, top_n=7)
    if not result:
        await update.message.reply_text("Нет данных для выбранных параметров или мало вакансий для анализа.")
    else:
        msg = (
            f"Для перехода с <b>{grade_from}</b> на <b>{grade_to}</b> по профессии <b>{title}</b> "
            f"на рынке чаще всего выделяют навыки:\n"
        )
        for i, (skill, delta, freq) in enumerate(result, 1):
            msg += f"{i}. <b>{skill}</b> (+{delta} вакансий, всего {freq})\n"
        await update.message.reply_text(msg, parse_mode="HTML")

    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END

ANALYZE_DESC, ANALYZE_CITY, ANALYZE_SKILLS, ANALYZE_SALARY = range(4)

async def analyze_start(update, context):
    await update.message.reply_text(
        "Опишите вакансию или ваш опыт работы (например: Python-разработчик, поддержка веб-сервиса, автоматизация бизнес-процессов):"
    )
    return ANALYZE_DESC

async def analyze_get_city(update, context):
    context.user_data['description'] = update.message.text.strip()
    await update.message.reply_text("В каком городе или регионе расположена вакансия (или где вы работаете)?")
    return ANALYZE_CITY

async def analyze_get_skills(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "Перечислите ключевые навыки через запятую (например: Python, Django, SQL, коммуникабельность):"
    )
    return ANALYZE_SKILLS

#async def analyze_get_grade(update, context):
#    context.user_data['skills'] = [s.strip() for s in update.message.text.split(',')]
#    await update.message.reply_text(
#        "Укажите уровень (junior, middle, senior, lead). Если не важен — напишите ‘-’ или оставьте пустым."
#    )
 #   return SKILLS_GRADE

async def analyze_get_salary(update, context):
    context.user_data['skills'] = [s.strip() for s in update.message.text.split(',')]
    await update.message.reply_text(
        "Укажите вашу зарплату (если хотите сравнить с рынком). Если не хотите — напишите ‘-’ или оставьте пустым."
    )
    return ANALYZE_SALARY

async def analyze_finish(update, context):
    description = context.user_data['description']
    city = context.user_data['city']
    skills = context.user_data['skills']
    salary_input = update.message.text.strip()
    try:
        salary_rub = int(salary_input.replace(' ', '').replace('руб', '')) if salary_input and salary_input != '-' else None
    except ValueError:
        salary_rub = None

    area_id = get_area_id_by_city(city) or 0

    vac = {
        'title': description[:40],          # для title
        'area_id': area_id,
        'experience_hh': None,              # если хочешь — добавить шаг с опытом
        'skills': skills,
        'salary_rub': salary_rub
    }

    # Анализ
    result = compare_vacancy_to_market(vac)
    await update.message.reply_text(result)
    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END

TOP_CITY, TOP_TITLE, TOP_GRADE = range(3)

async def top_start(update, context):
    await update.message.reply_text("В каком городе или регионе вас интересуют вакансии?")
    return TOP_CITY

async def top_get_title(update, context):
    context.user_data['city'] = update.message.text.strip()
    await update.message.reply_text(
        "Введите ключевое слово или профессию (например: Python, маркетолог, менеджер по продажам):"
    )
    return TOP_TITLE

async def top_get_grade(update, context):
    context.user_data['title'] = update.message.text.strip()
    await update.message.reply_text(
        "Укажите уровень (junior, middle, senior, lead), если важно. Если не важно — напишите ‘-’ или оставьте пустым."
    )
    return TOP_GRADE

async def top_finish(update, context):
    city = context.user_data['city']
    title = context.user_data['title']
    grade_input = update.message.text.strip().lower()

    area_id = get_area_id_by_city(city) or 0

    grade_map = {'junior': 0, 'middle': 1, 'senior': 2, 'lead': 3}
    grade = grade_map.get(grade_input, None) if grade_input and grade_input != '-' else None

    # Получаем топ-вакансий (твоя функция должна возвращать DataFrame/список вакансий)
    df = top_vacancies(area_name=city, keyword=title, grade=grade)

    if df.empty:
        await update.message.reply_text("Не найдено вакансий по заданным параметрам.")
        await update.message.reply_text(MAIN_MENU_TEXT)
        return ConversationHandler.END

    msg = f"<b>Топ-5 вакансий по вашему запросу:</b>\nГород: {city}\nПрофессия: {title}\n"
    if grade_input and grade_input != '-':
        msg += f"Грейд: {grade_input}\n"
    msg += "\n"
    for i, row in df.iterrows():
        salary = f"{int(row['salary_rub']):,}".replace(',', ' ') if row['salary_rub'] else "-"
        msg += (
            f"{i+1}. <b>{row['title']}</b> ({row['employer']})\n"
            f"Зарплата: {salary} руб.\n"
            f"Город: {city}\n"
            "— — —\n"
        )
    await update.message.reply_text(msg, parse_mode="HTML")
    await update.message.reply_text(MAIN_MENU_TEXT)
    return ConversationHandler.END


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Напиши /salary, /grade, /skills или /analyze для получения аналитики по рынку труда.\n"
        "/salary — Команда позволяет быстро получить прогноз средней зарплаты по любой вакансии.\n"
        "Подходит и работодателям, и соискателям: можно оценить “адекватность” зарплаты для предложенной позиции на рынке.\n"

    )

def main():

    app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    conv_salary = ConversationHandler(
        entry_points=[CommandHandler("salary", salary_start)],
        states={
            SALARY_DESC: [MessageHandler(filters.TEXT & ~filters.COMMAND, salary_get_city)],
            SALARY_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, salary_get_skills)],
            SALARY_SKILLS: [MessageHandler(filters.TEXT & ~filters.COMMAND, salary_get_grade)],
            SALARY_GRADE: [MessageHandler(filters.TEXT & ~filters.COMMAND, salary_finish)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    conv_skills = ConversationHandler(
       entry_points=[CommandHandler("skills", skills_start)],
        states={
            SKILLS_TITLE: [MessageHandler(filters.TEXT & ~filters.COMMAND, skills_get_city)],
            SKILLS_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, skills_get_grade)],
            SKILLS_GRADE: [MessageHandler(filters.TEXT & ~filters.COMMAND, skills_finish)],
        },
            fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    conv_topjobs = ConversationHandler(
        entry_points=[CommandHandler("top", top_start)],
        states={
            TOP_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, top_get_title)],
            TOP_TITLE: [MessageHandler(filters.TEXT & ~filters.COMMAND, top_get_grade)],
            TOP_GRADE: [MessageHandler(filters.TEXT & ~filters.COMMAND, top_finish)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    conv_grade = ConversationHandler(
        entry_points=[CommandHandler("grade", grade_start)],
        states={
            GRADE_DESC: [MessageHandler(filters.TEXT & ~filters.COMMAND, grade_get_city)],
            GRADE_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, grade_get_skills)],
            GRADE_SKILLS: [MessageHandler(filters.TEXT & ~filters.COMMAND, grade_finish)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    # Добавить handler:
    conv_nextskills = ConversationHandler(
        entry_points=[CommandHandler("nextskills", nextskills_start)],
        states={
            NEXTSKILLS_TITLE: [MessageHandler(filters.TEXT & ~filters.COMMAND, nextskills_get_city)],
            NEXTSKILLS_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, nextskills_get_from)],
            NEXTSKILLS_FROM: [MessageHandler(filters.TEXT & ~filters.COMMAND, nextskills_get_to)],
            NEXTSKILLS_TO: [MessageHandler(filters.TEXT & ~filters.COMMAND, nextskills_finish)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )
    conv_analyze = ConversationHandler(
        entry_points=[CommandHandler("analyze", analyze_start)],
        states={
            ANALYZE_DESC: [MessageHandler(filters.TEXT & ~filters.COMMAND, analyze_get_city)],
            ANALYZE_CITY: [MessageHandler(filters.TEXT & ~filters.COMMAND, analyze_get_skills)],
            ANALYZE_SKILLS: [MessageHandler(filters.TEXT & ~filters.COMMAND, analyze_get_salary)],
            ANALYZE_SALARY: [MessageHandler(filters.TEXT & ~filters.COMMAND, analyze_finish)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(conv_salary)
    app.add_handler(conv_skills)
    app.add_handler(conv_analyze)
    app.add_handler(conv_topjobs)
    app.add_handler(conv_grade)
    app.add_handler(conv_nextskills)


    print("Бот запущен!")
    app.run_polling()

if __name__ == "__main__":
    main()


Overwriting src/bot/bot.py


In [3]:
!python src/bot/bot.py

SyntaxError: invalid syntax (<ipython-input-3-1721720889>, line 1)

In [ ]:
!kill 693
